# Virus Data Collector

### Import Libraries

In [1]:
import requests
import pandas as pd
import csv
import json
import time
from typing import List, Dict
from datetime import datetime
import os

In [2]:
# ===== CONFIGURATION =====
API_KEY = '2117ff9ed05bbfde342deec3c7e417fa98cd4068adb477f43ac3c1d58e29431a'  # Your API key
DELAY_BETWEEN_REQUESTS = 16  # seconds (free tier: 4 req/min)

headers = {"x-apikey": API_KEY}

In [3]:

# ===== HELPER FUNCTIONS =====

# Read hash values from a CSV file
def read_hash_file(file_name: str, hash_col: str = "hash") -> List[Dict]:
    # List to store hash info
    hash_data = []
    try:
        with open(file_name, newline='', encoding='utf-8') as fh:
            reader = csv.DictReader(fh)  # Open CSV as dictionary reader
            
            # Check if headers exist
            if reader.fieldnames is None:
                raise ValueError("CSV file appears to have no header row.")
            
            # Ensure hash column exists
            if hash_col not in reader.fieldnames:
                raise ValueError(f"Column '{hash_col}' not found in CSV header: {reader.fieldnames}")

            # Iterate each row and extract hash and related info
            for row in reader:
                raw_hash = row.get(hash_col, "").strip().lower()
                # Skip empty or example hashes
                if raw_hash and raw_hash != "example_hash_here":
                    hash_data.append({
                        'hash': raw_hash,
                        'family': row.get('malware_family', 'Unknown'),
                        'source': row.get('source', 'Unknown')
                    })
        
        # Print number of hashes loaded
        print(f"✓ Loaded {len(hash_data)} hashes from {file_name}")
        return hash_data
    
    # Handle file not found error
    except FileNotFoundError:
        print(f"✗ Error: File '{file_name}' not found!")
        return []
    # Handle other exceptions
    except Exception as e:
        print(f"✗ Error reading file: {e}")
        return []



In [4]:

# Get the file report from VirusTotal API given a hash
def get_file_report(hash_val: str) -> Dict:
    url = f"https://www.virustotal.com/api/v3/files/{hash_val}"
    try:
        response = requests.get(url, headers=headers)
        # Successful response
        if response.status_code == 200:
            return {'success': True, 'data': response.json()}
        # Hash not found on VirusTotal
        elif response.status_code == 404:
            return {'success': False, 'error': 'Hash not found on VirusTotal'}
        # Other errors
        else:
            return {'success': False, 'error': f'Error {response.status_code}'}
    except Exception as e:
        return {'success': False, 'error': str(e)}


In [5]:

# Get behavior/sandbox report from VirusTotal API for a given hash
def get_behavior_report(hash_val: str) -> Dict:
    url = f"https://www.virustotal.com/api/v3/files/{hash_val}/behaviour_summary"
    try:
        response = requests.get(url, headers=headers)
        # Successful response
        if response.status_code == 200:
            return {'success': True, 'data': response.json()}
        # No behavior data available for hash
        elif response.status_code == 404:
            return {'success': False, 'error': 'No behavior data available'}
        # Other errors
        else:
            return {'success': False, 'error': f'Error {response.status_code}'}
    except Exception as e:
        return {'success': False, 'error': str(e)}

# Extract key behavioral indicators from the sandbox report JSON
def extract_behavioral_indicators(behavior_data: Dict) -> Dict:
    # Prepare empty lists to collect indicators
    indicators = {
        'processes_created': [],
        'files_written': [],
        'files_deleted': [],
        'registry_keys_set': [],
        'registry_keys_deleted': [],
        'dns_lookups': [],
        'ip_traffic': [],
        'http_conversations': [],
        'command_executions': [],
        'mutexes_created': [],
        'services_created': [],
        'mitre_techniques': []
    }
    
    # If behavior data unsuccessful, return empty indicators
    if not behavior_data.get('success'):
        return indicators
    
    # Extract attributes from response JSON
    data = behavior_data.get('data', {}).get('data', {}).get('attributes', {})
    
    # Extract various indicators if they exist in data
    if 'processes_created' in data:
        indicators['processes_created'] = data['processes_created']
    if 'files_written' in data:
        indicators['files_written'] = data['files_written']
    if 'files_deleted' in data:
        indicators['files_deleted'] = data['files_deleted']
    if 'registry_keys_set' in data:
        indicators['registry_keys_set'] = data['registry_keys_set']
    if 'registry_keys_deleted' in data:
        indicators['registry_keys_deleted'] = data['registry_keys_deleted']
    if 'dns_lookups' in data:
        indicators['dns_lookups'] = [lookup.get('hostname', '') for lookup in data['dns_lookups']]
    if 'ip_traffic' in data:
        indicators['ip_traffic'] = [f"{ip.get('destination_ip', '')}:{ip.get('destination_port', '')}" 
                                     for ip in data['ip_traffic']]
    if 'http_conversations' in data:
        indicators['http_conversations'] = [conv.get('url', '') for conv in data['http_conversations']]
    if 'command_executions' in data:
        indicators['command_executions'] = data['command_executions']
    if 'mutexes_created' in data:
        indicators['mutexes_created'] = data['mutexes_created']
    if 'mitre_attack_techniques' in data:
        indicators['mitre_techniques'] = data['mitre_attack_techniques']
    
    return indicators


In [ ]:
# Collect data for all malware hash samples and save results to files
def collect_sample_data(hash_data_list: List[Dict], output_dir: str = "output"):
    # Create output directories if they do not exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(f"{output_dir}/raw_responses", exist_ok=True)
    
    results = []
    total = len(hash_data_list)
    
    # Print start info and estimated total time
    print(f"\n{'='*60}")
    print(f"Starting data collection for {total} samples...")
    print(f"Estimated time: {(total * DELAY_BETWEEN_REQUESTS) / 60:.1f} minutes")
    print(f"{'='*60}\n")
    
    # Loop through hashes one by one
    for idx, hash_info in enumerate(hash_data_list, 1):
        hash_val = hash_info['hash']
        family = hash_info['family']
        
        print(f"[{idx}/{total}] Processing: {hash_val[:16]}... ({family})")
        
        # Get file report from VirusTotal
        print(f"  → Fetching file report...")
        file_report = get_file_report(hash_val)
        
        # If file report failed, record failure and continue to next item
        if not file_report['success']:
            print(f"  ✗ {file_report['error']}")
            results.append({
                'hash': hash_val,
                'family': family,
                'status': 'failed',
                'error': file_report['error']
            })
            time.sleep(DELAY_BETWEEN_REQUESTS)
            continue
        
        # Wait to respect rate limit
        time.sleep(DELAY_BETWEEN_REQUESTS)
        
        # Get behavioral report from VirusTotal
        print(f"  → Fetching behavior report...")
        behavior_report = get_behavior_report(hash_val)
        
        # Save raw JSON responses to files
        with open(f"{output_dir}/raw_responses/{hash_val}_file.json", 'w') as f:
            json.dump(file_report, f, indent=2)
        with open(f"{output_dir}/raw_responses/{hash_val}_behavior.json", 'w') as f:
            json.dump(behavior_report, f, indent=2)
        
        # Extract behavioral indicators from sandbox data
        indicators = extract_behavioral_indicators(behavior_report)
        
        # Prepare result dictionary to store counts and info
        result = {
            'hash': hash_val,
            'family': family,
            'source': hash_info['source'],
            'status': 'success',
            'detection_ratio': None,
            'first_seen': None,
            'last_seen': None,
            'processes_count': len(indicators['processes_created']),
            'files_written_count': len(indicators['files_written']),
            'files_deleted_count': len(indicators['files_deleted']),
            'registry_keys_set_count': len(indicators['registry_keys_set']),
            'dns_lookups_count': len(indicators['dns_lookups']),
            'ip_connections_count': len(indicators['ip_traffic']),
            'http_requests_count': len(indicators['http_conversations']),
            'mutexes_count': len(indicators['mutexes_created']),
            'mitre_techniques_count': len(indicators['mitre_techniques']),
            'mitre_techniques': ', '.join([t.get('id', '') for t in indicators['mitre_techniques']]),
            'collected_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        # Get detection stats and timestamps from file report, if available
        if file_report['success']:
            attrs = file_report['data'].get('data', {}).get('attributes', {})
            stats = attrs.get('last_analysis_stats', {})
            result['detection_ratio'] = f"{stats.get('malicious', 0)}/{sum(stats.values())}"
            result['first_seen'] = attrs.get('first_submission_date', 'N/A')
            result['last_seen'] = attrs.get('last_analysis_date', 'N/A')
        
        # Add current result to results list
        results.append(result)
        
        print(f"  ✓ Success! Detections: {result['detection_ratio']}, "
              f"MITRE Techniques: {result['mitre_techniques_count']}")
        
        # Save progress every 5 samples
        if idx % 5 == 0:
            df_temp = pd.DataFrame(results)
            df_temp.to_csv(f"{output_dir}/analysis_results_partial.csv", index=False)
            print(f"\n  💾 Progress saved ({idx}/{total} completed)\n")
        
        # Wait before next request (except after last one)
        if idx < total:
            print(f"  ⏳ Waiting {DELAY_BETWEEN_REQUESTS}s (rate limit)...\n")
            time.sleep(DELAY_BETWEEN_REQUESTS)
    
    # Save final results as CSV and JSON
    df_final = pd.DataFrame(results)
    df_final.to_csv(f"{output_dir}/analysis_results.csv", index=False)
    df_final.to_json(f"{output_dir}/analysis_results.json", orient="records", indent=2)
    
    # Summary output to console
    print(f"\n{'='*60}")
    print(f"✓ Data collection complete!")
    print(f"  Total samples: {total}")
    print(f"  Successful: {sum(1 for r in results if r['status'] == 'success')}")
    print(f"  Failed: {sum(1 for r in results if r['status'] == 'failed')}")
    print(f"\nResults saved to:")
    print(f"  - {output_dir}/analysis_results.csv")
    print(f"  - {output_dir}/analysis_results.json")
    print(f"{'='*60}\n")
    
    return results

# Generate a summary report based on collected results
def generate_summary_report(results: List[Dict], output_dir: str = "output"):
    # Convert results list to dataframe for analysis
    df = pd.DataFrame(results)
    # Filter only successful samples
    successful = df[df['status'] == 'success']
    
    print("\n" + "="*60)
    print("COLLECTION SUMMARY REPORT")
    print("="*60)
    
    # Print basic summary counts
    print(f"\nTotal samples processed: {len(results)}")
    print(f"Successful collections: {len(successful)}")
    print(f"Failed collections: {len(results) - len(successful)}")
    
    if len(successful) > 0:
        # Show count by malware family
        print(f"\nSamples by Malware Family:")
        family_counts = successful['family'].value_counts()
        for family, count in family_counts.items():
            print(f"  {family}: {count}")
        
        # Show averages of behavioral indicators
        print(f"\nBehavioral Indicators Summary:")
        print(f"  Avg processes created: {successful['processes_count'].mean():.1f}")
        print(f"  Avg files written: {successful['files_written_count'].mean():.1f}")
        print(f"  Avg registry modifications: {successful['registry_keys_set_count'].mean():.1f}")
        print(f"  Avg DNS lookups: {successful['dns_lookups_count'].mean():.1f}")
        print(f"  Avg network connections: {successful['ip_connections_count'].mean():.1f}")
        
        # MITRE ATT&CK technique coverage info
        print(f"\nMITRE ATT&CK Coverage:")
        print(f"  Samples with MITRE techniques: {(successful['mitre_techniques_count'] > 0).sum()}")
        print(f"  Total unique techniques identified: {successful['mitre_techniques'].nunique()}")
    
    print("="*60 + "\n")



VirusTotal Malware Data Collection Script

✗ Error: File './output/hash_signature_output.csv' not found!
No hashes found. Please check your input file.
Found 0 hashes to process

Preview of samples:


In [ ]:
# ===== MAIN EXECUTION START =====
if __name__ == "__main__":
    print("\n" + "="*60)
    print("VirusTotal Malware Data Collection Script")
    print("="*60 + "\n")
    
    # Define input and output paths
    INPUT_FILE = "./output/hash_signature_output.csv"
    OUTPUT_DIR = "./output"
    
    # Read hashes to process
    hash_data = read_hash_file(INPUT_FILE)
    
    # Exit if no hashes found
    if not hash_data:
        print("No hashes found. Please check your input file.")
        exit(1)
    
    # Show number of hashes and preview first 3
    print(f"Found {len(hash_data)} hashes to process")
    print("\nPreview of samples:")
    for i, h in enumerate(hash_data[:3], 1):
        print(f"  {i}. {h['hash'][:16]}... ({h['family']})")
    if len(hash_data) > 3:
        print(f"  ... and {len(hash_data) - 3} more")
    
    # Ask user to confirm before starting data collection
    response = input("\nProceed with data collection? (yes/no): ").strip().lower()
    
    if response == 'yes':
        # Collect data and save to files
        results = collect_sample_data(hash_data, OUTPUT_DIR)
        # Print summary report
        generate_summary_report(results, OUTPUT_DIR)
        print("\n✓ All done! Check the output folder for results.")
    else:
        print("\nCollection cancelled.")